# Find ships closest to observations

In [ ]:
import pandas as pd
import utils

In [ ]:
aisPath = '~/fairsea_data/ais2018_chemical_tanker.csv'
obsPath = '~/fairsea_data/pollution-obs-2018.csv'

## Import data from CSVs

In [ ]:
df_ais = pd.read_csv(aisPath, parse_dates=["Timestamp_datetime"])
df_observations = pd.read_csv(obsPath, delimiter=';', header=None ,names=["date", "time", "Latitude", "Longitude"])

# Convert observation date and time columns into formatted datetime column
df_observations['datetime'] = df_observations['date'] + ' ' + df_observations['time']
df_observations = df_observations.drop(columns=['date', 'time'])
df_observations.datetime = pd.to_datetime(df_observations.datetime, format='%m/%d/%Y %H:%M')

print(f"Imported data for {df_ais.IMO.nunique()} ships and {len(df_observations.index)} observations")

## Find ships closest to observations

In [ ]:
dt = pd.Timedelta(2, "h")

In [ ]:
df_observations['closestShip'] = df_observations.apply(lambda o: utils.findClosestShips(o.squeeze(), df_ais, dt, 3), axis=1)

In [ ]:
trajs = [
    [
    df_ais[
        (df_ais["IMO"] == imo) &
        (df_ais["Timestamp_datetime"] > row['datetime'] - dt) &
        (df_ais["Timestamp_datetime"] <= row['datetime'] + dt)
    ]
    for imo in row['closestShip'] if imo != 0
    ] for _, row in df_observations.iterrows()
]

## Draw interactive map

In [ ]:
utils.mapObservations(df_observations, trajs)

In [ ]:
utils.findClosestShips(df_observations.head(1).squeeze(), df_ais, dt, 10)